In [1]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
import random
import pickle
from nltk.tokenize import RegexpTokenizer
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dense, Activation
from tensorflow.keras.optimizers import RMSprop

In [3]:
text_df = pd.read_csv("fake_or_real_news.csv")


In [4]:
text= list(text_df.text.values)

joined_text = " ".join(text)



In [5]:
partial_text = joined_text[:10000]


In [6]:
tokenizer = RegexpTokenizer(r"\w+")
tokens = tokenizer.tokenize(partial_text.lower())

In [7]:
unique_tokens = np.unique(tokens)
unique_token_index = {token: idx for idx, token in enumerate(unique_tokens)}

In [8]:
n_words = 10
input_words = []
next_words = []

for i in range(len(tokens)- n_words):
    input_words.append(tokens[i:i + n_words])
    next_words.append(tokens[i + n_words])

In [84]:
input_words

[['daniel',
  'greenfield',
  'a',
  'shillman',
  'journalism',
  'fellow',
  'at',
  'the',
  'freedom',
  'center'],
 ['greenfield',
  'a',
  'shillman',
  'journalism',
  'fellow',
  'at',
  'the',
  'freedom',
  'center',
  'is'],
 ['a',
  'shillman',
  'journalism',
  'fellow',
  'at',
  'the',
  'freedom',
  'center',
  'is',
  'a'],
 ['shillman',
  'journalism',
  'fellow',
  'at',
  'the',
  'freedom',
  'center',
  'is',
  'a',
  'new'],
 ['journalism',
  'fellow',
  'at',
  'the',
  'freedom',
  'center',
  'is',
  'a',
  'new',
  'york'],
 ['fellow',
  'at',
  'the',
  'freedom',
  'center',
  'is',
  'a',
  'new',
  'york',
  'writer'],
 ['at',
  'the',
  'freedom',
  'center',
  'is',
  'a',
  'new',
  'york',
  'writer',
  'focusing'],
 ['the',
  'freedom',
  'center',
  'is',
  'a',
  'new',
  'york',
  'writer',
  'focusing',
  'on'],
 ['freedom',
  'center',
  'is',
  'a',
  'new',
  'york',
  'writer',
  'focusing',
  'on',
  'radical'],
 ['center',
  'is',
  'a',
  

In [9]:
X = np.zeros((len(input_words), n_words, len(unique_tokens)), dtype= bool)
y = np.zeros((len(next_words), len(unique_tokens)), dtype = bool)

In [10]:
for i,words in enumerate(input_words):
    for j, word in enumerate(words):
        X[i,j, unique_token_index[word]] = 1
    y[i, unique_token_index[next_words[i]]] = 1

In [11]:
model = Sequential()
model.add(LSTM(128, input_shape=(n_words, len(unique_tokens)), return_sequences = True))
model.add(LSTM(128))
model.add(Dense(len(unique_tokens)))
model.add(Activation("softmax"))

In [12]:
model.compile(loss = "categorical_crossentropy", optimizer = tf.keras.optimizers.legacy.RMSprop(learning_rate=0.001), metrics= ['accuracy'])
model.fit(X, y, batch_size = 128, epochs = 30, shuffle = True)

Epoch 1/30
14/14 [==============================] - 1s 23ms/step - loss: 6.2306 - accuracy: 0.0435
Epoch 2/30
14/14 [==============================] - 0s 27ms/step - loss: 5.7763 - accuracy: 0.0618
Epoch 3/30
14/14 [==============================] - 0s 27ms/step - loss: 5.7244 - accuracy: 0.0618
Epoch 4/30
14/14 [==============================] - 0s 27ms/step - loss: 5.6839 - accuracy: 0.0618
Epoch 5/30
14/14 [==============================] - 0s 27ms/step - loss: 5.6515 - accuracy: 0.0618
Epoch 6/30
14/14 [==============================] - 0s 27ms/step - loss: 5.6242 - accuracy: 0.0618
Epoch 7/30
14/14 [==============================] - 0s 28ms/step - loss: 5.5785 - accuracy: 0.0618
Epoch 8/30
14/14 [==============================] - 0s 28ms/step - loss: 5.5365 - accuracy: 0.0641
Epoch 9/30
14/14 [==============================] - 0s 30ms/step - loss: 5.5053 - accuracy: 0.0618
Epoch 10/30
14/14 [==============================] - 0s 29ms/step - loss: 5.4619 - accuracy: 0.0669
Epoch 11/

In [14]:
model.save("mymodel.keras")

In [15]:
model = load_model("mymodel.keras")

In [16]:
def predict_next_word(input_text, n_best):
    input_text = input_text.lower()
    X = np.zeros((1, n_words, len(unique_tokens)))
    for i, word in enumerate(input_text.split()):
        X[0, i, unique_token_index[word]] = 1
        
        
    predictions = model.predict(X)[0]
    return np.argpartition(predictions, n_best)[-n_best:]

In [17]:
possible = predict_next_word("He will have to look into this thing and he", 5)

1/1 [==============================] - 0s 269ms/step


In [18]:
print([unique_tokens[idx] for idx in possible])

['years', 'about', 'you', 'a', 'zero']


In [22]:
def generate_text(input_text, text_length, creativity= 3):
    word_sequence = input_text.split()
    current = 0
    for _ in range(text_length):
        sub_sequence = " ".join(tokenizer.tokenize(" ".join(word_sequence).lower())[current:current+n_words])
        try:
            choice = unique_token[random.choice(predict_next_wword(sub_sequence, creativity))]
            
        except:
            choice = random.choice(unique_tokens)
            
        word_sequence.append(choice)
        current += 1
        
    return  " ".join(word_sequence)
    

In [23]:
generate_text("He will have to look into this thing and he", 100, 5)

'He will have to look into this thing and he trump gone revelation scene he word tape principled sexism assume on tells declaring hospital much rating running being why statement currently train preemptively him breathing pieces exposing better plan sudden trying ryan continue radical down energy were women was public misguided awkward stored smoke world film information promising appeaser right freedom violating cnn refused speaks ride down nobody democrats compared either attacking smoke bigotry days exactly gave than poses major for attacked lot she unapologetic principled breathing their might associates defending respond https stumbleupon making harry decides moment playing twitter impropriety war throughout bathroom daniel waking assaults elect fight belief'